In [3]:
#
# The MultiPole Expansion Software (MPES)
#
#             by
#
#        Fredrik Knapskog
#
# Trondheim, 28-May-2021
#
#     --- o0o ---
#
# Routines:  get_spherical_coords
#            get_l
#            get_m
#            H
#            get_C
#            get_b
#            get_k
#            get_B
#            get_B_0
#            get_p
#            get_i1
#            get_j1
#            get_length
#            E_angles
#            susceptibilities
#            s_polarization
#            p_polarization
#            is_outside
#            coord_table
#            spherical_harm
#            potential_grid
#            field_grid
#
#            main            
#


# =========================================================================
#  Importing libraries
# =========================================================================

import numpy as np
import scipy
from scipy.special import sph_harm, binom
from scipy.sparse.linalg import gmres #Iterative solver
from time import *
from pyshtools import expand
from numba import jit, prange
# import os
# os.chdir('C:\\Users\\Xx-Fr\\pysopra1')
# from pysopra import EpsilonSOPRA, micron2eV, eV2micron

# --------------------------------------------------------------------------
# --- end importing libraries
# --------------------------------------------------------------------------

# =========================================================================
#  Implementing functions
# =========================================================================

#Returns five coordinates. The first two are the polar and azimuthal 
#angles of point p_2 relative to point p_1. Third is the polar angle
#of p_2s image multipole relative to p_1. Fourth is the distance between
#p_1 and p_2 and fifth is the distance between p_2s image multipole
#and p_1.
@jit(nopython=True)
def get_spherical_coords(p_1, p_2):
    polar_angle = np.arctan2(np.sqrt((p_2[0]-p_1[0])**2 + (p_2[1]-p_1[1])**2), p_2[2]-p_1[2])
    azimuthal_angle = np.arctan2(p_2[1]-p_1[1], p_2[0]-p_1[0])
    image_polar_angle = np.arctan2(np.sqrt((p_2[0]-p_1[0])**2 + (p_2[1]-p_1[1])**2), -p_2[2]-p_1[2])
    R = np.sqrt((p_2[0]-p_1[0])**2 + (p_2[1]-p_1[1])**2 + (p_2[2]-p_1[2])**2)
    image_R = np.sqrt((p_2[0]-p_1[0])**2 + (p_2[1]-p_1[1])**2 + (p_2[2]+p_1[2])**2)
    return polar_angle, azimuthal_angle, image_polar_angle, R, image_R

# --------------------------------------------------------------------------

#Returns the index l from matrix index ind (with numba)
@jit(nopython=True)
def get_l(ind):
    return np.sqrt(ind+1)//1

# --------------------------------------------------------------------------

#Returns the index m from matrix index ind (with numba)
@jit(nopython=True)
def get_m(ind):
    l = get_l(ind)
    return ind + 1 -l*(l+1)

# --------------------------------------------------------------------------

#Returns the index l from matrix index ind (without numba)
def get_l1(ind):
    return np.sqrt(ind+1)//1

# --------------------------------------------------------------------------

#Returns the index m from matrix index ind (without numba)
def get_m1(ind):
    l = get_l(ind)
    return ind + 1 -l*(l+1)

# --------------------------------------------------------------------------

#Computes H(l_j, m_j | l_i, m_i) for an array of indices k
def H(k, M):
    k1 = k//M
    l1 = k%M
    l_i, m_i, l_j, m_j = get_l1(l1), get_m1(l1), get_l1(k1), get_m1(k1)
    l = l_i + l_j
    m = m_i - m_j
    return np.sqrt(4*np.pi)*(-1)**(l_i+m_j) * np.sqrt((2*l_i+1)/((2*l_j+1)*(2*l+1))) * np.sqrt(binom(l+m, l_i+m_i)*binom(l-m, l_j+m_j))

# --------------------------------------------------------------------------

#Computes H(l_j, m_j | l_i, m_i) for single input
def H1(l_j, m_j, l_i, m_i):
    l = l_i + l_j
    m = m_i - m_j
    return np.sqrt(4*np.pi)*(-1)**(l_i+m_j) * np.sqrt((2*l_i+1)/((2*l_j+1)*(2*l+1))) * np.sqrt(binom(l+m, l_i+m_i)*binom(l-m, l_j+m_j))

# --------------------------------------------------------------------------

#Builds C matrix
@jit(nopython=True, parallel = True)
def get_C(C, L_parallel, L_orthogonal, M, N_cell, lattice_points, centre_point, lattice_vectors, epsilon, r, beta, epsilon_plus, coords, spherical_harmonics, image_spherical_harmonics, Hs, k_vec, b_x, b_y, alpha):
    M_parallel = (L_parallel+1)**2 - 1
    for k in range(N_cell*M):
        j, k1 = k//M, k%M
        l_j, m_j = get_l(k1), get_m(k1)
        C[k,k] += r[j]**(-2*l_j-1)*(l_j*epsilon[j] + epsilon_plus*(l_j+1))/(l_j*(epsilon[j] - epsilon_plus))
        for point in range(lattice_points):
            phase_factor = np.exp(1j*(k_vec[0]*lattice_vectors[point, 0] + k_vec[1]*lattice_vectors[point, 1]))
            if point != centre_point:
                for l in prange(N_cell*M_parallel):
                    i, l1 = l//M_parallel, l%M_parallel
                    l_i, m_i = get_l(l1), get_m(l1)
                    l2, m2 = l_i+l_j, m_i-m_j
                    ind = int(l2*(l2+1)/2+np.abs(m2))
                    element1 = Hs[k1,l1]/coords[j,i,point,3]**(l2+1)
                    element2 = Hs[k1,l1]/coords[j,i,point,4]**(l2+1)
                    if m2 >= 0:
                        C[k,M*i+l1] += phase_factor * (element1*spherical_harmonics[j,i,point,ind] + element2*(-1)**(l_i+m_i)*beta * image_spherical_harmonics[j,i,point,ind])
                    else:
                        C[k,M*i+l1] += phase_factor * (element1*(-1)**m2 * np.conj(spherical_harmonics[j,i,point,ind]) + element2*(-1)**(l_i+m_i +m2)*beta * np.conj(image_spherical_harmonics[j,i,point,ind]))
            else:
                for l in prange(N_cell*M):
                    i, l1 = l//M, l%M
                    l_i, m_i = get_l(l1), get_m(l1)
                    if i != j:
                        if l_i <= L_parallel:
                            l2, m2 = l_i+l_j, m_i-m_j
                            ind = int(l2*(l2+1)/2+np.abs(m2))
                            element1 = Hs[k1,l1]/coords[j,i,point,3]**(l2+1)
                            element2 = Hs[k1,l1]/coords[j,i,point,4]**(l2+1)
                            if m2 >= 0:
                                C[k,l] += element1 * spherical_harmonics[j,i,point,ind] + element2*(-1)**(l_i+m_i)*beta * image_spherical_harmonics[j,i,point,ind]
                            else:
                                C[k,l] += element1*(-1)**m2 * np.conj(spherical_harmonics[j,i,point,ind]) + element2*(-1)**(l_i+m_i +m2)*beta * np.conj(image_spherical_harmonics[j,i,point,ind])
                    else:
                        if l_i <= L_orthogonal and m_j == m_i:
                            l2, m2 = l_i+l_j, m_i-m_j
                            ind = int(l2*(l2+1)/2)
                            C[k,l] += Hs[k1,l1] / coords[j,i,point,4]**(l2+1) * beta*(-1)**(l_i+m_i) * image_spherical_harmonics[j,i,point,ind]
    return C

# --------------------------------------------------------------------------

#Creates b-vector from electric field direction
def get_b(theta_0, phi_0, M, N):
    b = np.zeros(M, dtype = np.complex128)
    b[0] = np.sqrt(2*np.pi/3)*np.sin(theta_0)*np.exp(1j*phi_0)
    b[1] = np.sqrt(4*np.pi/3)*np.cos(theta_0)
    b[2] = -np.sqrt(2*np.pi/3)*np.sin(theta_0)*np.exp(-1j*phi_0)
    return np.tile(b,N)

# --------------------------------------------------------------------------

#Creates k-vector from incident wave direction and wave number
def get_k(omega, theta_0, phi_0, length_scale):
    return length_scale*omega/1.97e-7*np.sin(theta_0)*np.array([np.cos(phi_0), np.sin(phi_0), 0])

# --------------------------------------------------------------------------

#Computes B coefficients from solved A coefficients
@jit(nopython=True, parallel = True)
def get_B(M, N, A, b, beta, r, Hs, coords, spherical_harmonics, image_spherical_harmonics, L_parallel, L_orthogonal):
    B = np.zeros(M*N, dtype = np.complex128)
    for k in prange(M*N):
        j, k1 = k//M, k%M
        l_j, m_j = get_l(k1), get_m(k1)
        B[k] += -r[j]**(1-l_j)*b[k] + A[k]*r[j]**(-2*l_j-1)
        for l in prange(M*N):
            i, l1 = l//M, l%M
            l_i, m_i = get_l(l1), get_m(l1)
            if i != j and l_i <= L_parallel:
                l2, m2 = l_i+l_j, m_i-m_j
                ind = int(l2*(l2+1)/2+np.abs(m2))
                element1 = Hs[k1,l1]/coords[j,i,3]**(l2+1)
                element2 = Hs[k1,l1]/coords[j,i,4]**(l2+1)
                if m2 >= 0:
                    B[k] += A[l]*(element1*spherical_harmonics[j,i,ind] + element2*(-1)**(l_i+m_i)*beta * image_spherical_harmonics[j,i,ind])
                else:
                    B[k] += A[l]*(element1*(-1)**m2 * np.conj(spherical_harmonics[j,i,ind]) + element2*(-1)**(l_i+m_i +m2)*beta * np.conj(image_spherical_harmonics[j,i,ind]))
            elif i == j and l_i <= L_orthogonal and m_j == m_i:
                l2= l_i+l_j
                ind = int(l2*(l2+1)/2)
                B[k] += A[l]*Hs[k1,l1]/coords[j,j,4]**(l2+1) * beta*(-1)**(l_i+m_i)*image_spherical_harmonics[j, j, ind]
    return B

# --------------------------------------------------------------------------

#Computes the B coefficient for m = l = 0
def get_B_0(M, N, A, beta, r, coords, spherical_harmonics, image_spherical_harmonics, L_parallel, L_orthogonal, origin, b):
    B_0 = np.zeros(N, dtype = np.complex128)
    for j in range(N):
        for l in range(M*N):
            i, l1 = l//M, l%M
            l_i, m_i = get_l(l1), get_m(l1)
            H_0 = H1(0, 0, l_i, m_i)
            if i != j and l_i <= L_parallel:
                l2, m2 = l_i, m_i
                ind = int(l2*(l2+1)/2+np.abs(m2))
                element1 = H_0/coords[j,i,3]**(l2+1)
                element2 = H_0/coords[j,i,4]**(l2+1)
                if m2 >= 0:
                    B_0[j] += A[l]*(element1*spherical_harmonics[j,i,ind] + element2*(-1)**(l_i+m_i)*beta * image_spherical_harmonics[j,i,ind])
                else:
                    B_0[j] += A[l]*(element1*(-1)**m2 * np.conj(spherical_harmonics[j,i,ind]) + element2*(-1)**(l_i+m_i +m2)*beta * np.conj(image_spherical_harmonics[j,i,ind]))
            elif i == j and l_i <= L_orthogonal and m_i == 0:
                l2= l_i
                ind = int(l2*(l2+1)/2)
                B_0[j] += A[l]*H_0/coords[j,j,4]**(l2+1) * beta*(-1)**(l_i+m_i)*image_spherical_harmonics[j, j, ind]
        if j != origin:
            B_0[j] += -np.sqrt(4*np.pi)*coords[origin, j, 3]*(-b[0]*np.conj(spherical_harmonics[origin, j, 2]) + b[1]*spherical_harmonics[origin, j, 1] + b[2]*spherical_harmonics[origin, j, 2])
    return B_0

# --------------------------------------------------------------------------

#Compute dimensionless dipole moment from A coefficients
def get_p(A, i, M, r):
    k = i*M
    p_x = np.sqrt(3/(8*np.pi))*(A[k]-A[k+2])/r[i]**2
    p_y = -1j*np.sqrt(3/(8*np.pi))*(A[k]+A[k+2])/r[i]**2
    p_z = np.sqrt(3/(4*np.pi))*A[k+1]/r[i]**2
    return np.real(np.sqrt(p_x*np.conj(p_x) + p_y*np.conj(p_y) + p_z*np.conj(p_z)))

# --------------------------------------------------------------------------

#Finds all i1 lattice coordinates to be included
#at lattice coordinate j1
def get_i1(R, b_x, b_y, alpha, j1):
    R1 = np.abs(j1*np.sin(alpha)*b_y)
    if R1 < R:
        i1_2 = int(np.floor(1/b_x*(np.sqrt(R**2 - (j1*np.sin(alpha)*b_y)**2) - j1*np.cos(alpha)*b_y)))
        i1_1 = int(np.ceil(1/b_x*(-np.sqrt(R**2 - (j1*np.sin(alpha)*b_y)**2) - j1*np.cos(alpha)*b_y)))
        return np.arange(i1_1, i1_2+1)
    elif R1 == R:
        i1_1 = int(1/b_x*(-j1*np.cos(alpha)*b_y))
        return np.arange(i1_1, i1_1+1)        
    else:
        return np.array([])

# --------------------------------------------------------------------------
    
#Finds all lattice coordinates j1 to be included
#based on the interaction distance R and 
#lattice distance b_y
def get_j1(R, b_y, alpha):
    j1_1 = R//(b_y*np.sin(alpha))
    return np.arange(-j1_1, j1_1+1)

# --------------------------------------------------------------------------

#Returns number of lattice sites to be included
def get_length(R, b_x, b_y, alpha):
    length = 0
    j1s = get_j1(R, b_y, alpha)
    for j1 in j1s:
        length += len(get_i1(R, b_x, b_y, alpha, j1))
    return length

# --------------------------------------------------------------------------

#Returns the electric field angles for p- and s-polarized
#light from incident wave angles
def E_angles(theta_0, phi_0):
    return np.pi/2 - theta_0, phi_0, np.pi/2, np.pi/2 + phi_0

# --------------------------------------------------------------------------

#Computes the susceptibilities gamma and beta from A coefficients
def susceptibilities(A, M, theta_0, phi_0, epsilon_plus, d, rho):
    if np.abs(np.cos(theta_0)) < 1e-6:
        alpha_ort_0 = 0
        alpha_ort_10 = 0
        alpha_par_0 = -4*np.pi*epsilon_plus*np.mean(A[2::M]) / (np.sqrt(2*np.pi/3)*np.sin(theta_0)*np.exp(-1j*phi_0))
        alpha_par_10 = -4*np.pi*epsilon_plus*np.mean(A[6::M]) / (np.sqrt(6*np.pi/5)*np.sin(theta_0)*np.exp(-1j*phi_0))
    elif np.abs(np.sin(theta_0)) < 1e-6:
        alpha_par_0 = 0
        alpha_par_10 = 0
        alpha_ort_0 = 2*np.pi*epsilon_plus*np.mean(A[1::M]) / (np.sqrt(np.pi/3)*np.cos(theta_0))
        alpha_ort_10 = np.pi*epsilon_plus*np.mean(A[5::M]) / (np.sqrt(np.pi/5)*np.cos(theta_0))
    else:
        alpha_par_0 = -4*np.pi*epsilon_plus*np.mean(A[2::M]) / (np.sqrt(2*np.pi/3)*np.sin(theta_0)*np.exp(-1j*phi_0))
        alpha_ort_0 = 2*np.pi*epsilon_plus*np.mean(A[1::M]) / (np.sqrt(np.pi/3)*np.cos(theta_0))
        alpha_par_10 = -4*np.pi*epsilon_plus*np.mean(A[6::M]) / (np.sqrt(6*np.pi/5)*np.sin(theta_0)*np.exp(-1j*phi_0))
        alpha_ort_10 = np.pi*epsilon_plus*np.mean(A[5::M]) / (np.sqrt(np.pi/5)*np.cos(theta_0))
    gamma_e = rho*alpha_par_0
    beta_e = rho*alpha_ort_0/epsilon_plus**2
    return gamma_e, beta_e

# --------------------------------------------------------------------------

#Returns the reflection and transmission coefficients
#for s-polarized light
def s_polarization(theta, omega, gamma_e, epsilon_plus, epsilon_minus, scale):
    n_plus, n_minus = np.sqrt(epsilon_plus), np.sqrt(epsilon_minus)
    theta_t = np.arcsin(n_plus/n_minus*np.sin(theta))
    HC = 1.97e-7/scale
    cos_theta, cos_theta_t = np.cos(theta), np.cos(theta_t)
    r_s = n_plus*cos_theta - n_minus*cos_theta_t + 1j*(omega/HC)*gamma_e
    t_s = 4*(n_minus*cos_theta)**2
    D_s = n_plus*cos_theta + n_minus*cos_theta_t - 1j*(omega/HC)*gamma_e
    return r_s/D_s, t_s/D_s

# --------------------------------------------------------------------------

#Returns the reflection and transmission coefficients
#for p-polarized light
def p_polarization(theta, omega, gamma_e, epsilon_plus, epsilon_minus, beta_e, scale):
    n_plus, n_minus = np.sqrt(epsilon_plus), np.sqrt(epsilon_minus)
    theta_t = np.arcsin(n_plus/n_minus*np.sin(theta))
    HC = 1.97e-7/scale
    cos_theta, cos_theta_t = np.cos(theta), np.cos(theta_t)
    r_p = (n_minus*cos_theta - n_plus*cos_theta_t) * (1-(omega/(2*HC))**2 * epsilon_plus*gamma_e*beta_e*np.sin(theta)**2) - 1j*omega/HC*gamma_e*cos_theta*cos_theta_t + 1j*omega/HC*n_plus*n_minus*epsilon_plus * beta_e*np.sin(theta)**2
    t_p = 2*n_minus*cos_theta
    D_p = (n_minus*cos_theta + n_plus*cos_theta_t) * (1-(omega/(2*HC))**2 * epsilon_plus*gamma_e*beta_e*np.sin(theta)**2) - 1j*omega/HC*gamma_e*cos_theta*cos_theta_t - 1j*omega/HC*n_plus*n_minus*epsilon_plus * beta_e*np.sin(theta)**2
    return r_p/D_p, t_p/D_p

# --------------------------------------------------------------------------

#Determines whether a point is contained by a sphere or not.
#If contained then the sphere containing it is returned too.
@jit(nopython=True)
def is_outside(point, N, r, p):
    status = True
    sphere = -1
    for i in range(N):
        if np.sqrt((point[0]-p[i,0])**2 + (point[1]-p[i,1])**2 + (point[2]-p[i,2])**2) < r[i]:
            status = False
            sphere = i
            break
    return status, sphere

# --------------------------------------------------------------------------

#Returns the x and z coordinates for the nearfield grid.
#Also computes the five coordinates from get_spherical_coords
#for all the spheres relative to all the grid points, 
#and relative to all the image positions of the grid.
# @jit(nopython=True) #, parallel = True)
def coord_table(table, p, N, x0, x1, z0, z1, Nx, Nz, y):
    x = np.linspace(x0, x1, Nx)
    z = np.linspace(z0, z1, Nz)
    
    image_positions = p.copy().T
    image_positions[-1] *= -1
    image_positions = image_positions.T
    image_table = table.copy()
    
    for k in range(N):
        for i in prange(Nz):
            for j in range(Nx):
                point = np.array([x[j], y, z[i]])
                table[k,i,j,0], table[k,i,j,1], table[k,i,j,2], table[k,i,j,3], table[k,i,j,4] = get_spherical_coords(p[k], point)
                image_table[k,i,j,0], image_table[k,i,j,1], image_table[k,i,j,2], image_table[k,i,j,3], image_table[k,i,j,4] = get_spherical_coords(image_positions[k], point)
    return x, z, table, image_table

# --------------------------------------------------------------------------

#Computes the spherical harmonics for the angles given by the
#positions and the corresponding image positions on the grid
#from coord_table
def spherical_harm(coords, image_coords, L, N, Nx, Nz):
    spherical_harmonics = np.zeros([N, Nz, Nx, (L+2)*(L+1)//2], dtype = np.complex128)
    image_spherical_harmonics = np.zeros([N, Nz, Nx, (L+2)*(L+1)//2], dtype = np.complex128)
    for k in range(N):
        for i in range(Nz):
            for j in range(Nx):
                spherical_harmonics[k,i,j] = expand.spharm(L, np.real(coords[k,i,j,0]), np.real(coords[k,i,j,1]), normalization = 'ortho', kind = 'complex', csphase = -1, packed = True, degrees = False)[0]
                image_spherical_harmonics[k,i,j] = expand.spharm(L, np.real(image_coords[k,i,j,0]), np.real(image_coords[k,i,j,1]), normalization = 'ortho', kind = 'complex', csphase = -1, packed = True, degrees = False)[0]
                
    return spherical_harmonics, image_spherical_harmonics

# --------------------------------------------------------------------------

#Evaluates the complex scalar potential divided by the
#potential from the electric field from the incident
#wave at each point on the grid from coord_table
@jit(nopython=True, parallel = True)
def potential_grid(pot, origin, x, y, z, Nx, Nz, coord_table, image_coord_table, spherical_harmonics, image_spherical_harmonics, A, r, N, M, beta, b, positions, B, beta2, epsilon_plus, epsilon_minus, B0, theta):
    for i in prange(Nz):
        for j in prange(Nx):
            point = np.array([x[j], y, z[i]])
            outside, sphere = is_outside(point, N, r, positions)
            if outside:
                if point[2] >= 0:
                    pot[i, j] += coord_table[origin,i,j,3] * b[0]*np.conj(spherical_harmonics[origin,i,j,2])
                    pot[i, j] += -coord_table[origin,i,j,3] * b[1]*spherical_harmonics[origin,i,j,1]
                    pot[i, j] += -coord_table[origin,i,j,3] * b[2]*spherical_harmonics[origin,i,j,2]
                    for k in range(N*M):
                        j1, k1 = k//M, k%M
                        l = get_l(k1)
                        m = get_m(k1)
                        ind = int(l*(l+1)/2+np.abs(m))
                        if m >= 0:
                            pot[i, j] += A[k] * (coord_table[j1,i,j,3]**(-l-1) * spherical_harmonics[j1,i,j,ind] + (-1)**(l+m)*beta * image_coord_table[j1,i,j,3]**(-l-1) * image_spherical_harmonics[j1,i,j,ind])
                        else:
                            pot[i, j] += (-1)**m*A[k] * (coord_table[j1,i,j,3]**(-l-1) * np.conj(spherical_harmonics[j1,i,j,ind]) + (-1)**(l+m)*beta * image_coord_table[j1,i,j,3]**(-l-1) * np.conj(image_spherical_harmonics[j1,i,j,ind]))
                else:
                    pot[i, j] += coord_table[origin,i,j,3] * b[0]*np.conj(spherical_harmonics[origin,i,j,2])
                    pot[i, j] += -coord_table[origin,i,j,3] * b[1]*spherical_harmonics[origin,i,j,1] * epsilon_plus/epsilon_minus
                    pot[i, j] += -coord_table[origin,i,j,3] * b[2]*spherical_harmonics[origin,i,j,2]
                    pot[i, j] += positions[origin,2] * (epsilon_plus/epsilon_minus -1) * np.cos(theta)
                    for k in range(N*M):
                        j1, k1 = k//M, k%M
                        l = get_l(k1)
                        m = get_m(k1)
                        ind = int(l*(l+1)/2+np.abs(m))
                        if m >= 0:
                            pot[i, j] += beta2*A[k] * coord_table[j1,i,j,3]**(-l-1) * spherical_harmonics[j1,i,j,ind]
                        else:
                            pot[i, j] += beta2*(-1)**m*A[k] * coord_table[j1,i,j,3]**(-l-1) * np.conj(spherical_harmonics[j1,i,j,ind])
            else:
                pot[i, j] += np.sqrt(1/(4*np.pi))*B0[sphere]
                for k in range(M):
                    l = get_l(k)
                    m = get_m(k)
                    ind = int(l*(l+1)/2+np.abs(m))
                    if m >= 0:
                        pot[i, j] += B[sphere*M+k] * coord_table[sphere,i,j,3]**l * spherical_harmonics[sphere,i,j,ind]
                    else:
                        pot[i, j] += (-1)**m * B[sphere*M+k] * coord_table[sphere,i,j,3]**l * np.conj(spherical_harmonics[sphere,i,j,ind])
    return pot

# --------------------------------------------------------------------------

#Evaluates the length of the electric field divided by
#the electric field from the incident wave for 
#each point on the grid from coord_table
@jit(nopython=True) #, parallel = True)
def field_grid(E, origin, x, y, z, Nx, Nz, coord_table, image_coord_table, spherical_harmonics, image_spherical_harmonics, A, r, N, M, beta, b, positions, B, beta2, epsilon_plus, epsilon_minus):
    for i in range(Nz):
        for j in range(Nx):
            x1, y1, z1 = 0, 0, 0
            point = np.array([x[j], y, z[i]])
            outside, sphere = is_outside(point, N, r, positions)
            if outside:
                dSH0_theta = 1/np.tan(coord_table[origin,i,j,0]) * np.conj(spherical_harmonics[origin,i,j,2]) + np.sqrt(2)*np.exp(-1j*coord_table[origin,i,j,1]) * spherical_harmonics[origin,i,j,1]
                dSH1_theta = np.sqrt(2) * np.exp(-1j*coord_table[origin,i,j,1]) * spherical_harmonics[origin,i,j,2]
                dSH2_theta = spherical_harmonics[origin,i,j,2] / np.tan(coord_table[origin,i,j,0])
                dSH0_phi = 1j*np.conj(spherical_harmonics[origin,i,j,2])
                dSH2_phi = 1j*spherical_harmonics[origin,i,j,2]
                dSH0_r = -np.conj(spherical_harmonics[origin,i,j,2])
                dSH1_r = spherical_harmonics[origin,i,j,1]
                dSH2_r = spherical_harmonics[origin,i,j,2]
                if point[2] >= 0:
                    x1 += np.sin(coord_table[origin,i,j,0]) * np.cos(coord_table[origin,i,j,1]) * (b[0]*dSH0_r + b[1]*dSH1_r + b[2]*dSH2_r)
                    x1 += np.cos(coord_table[origin,i,j,0]) * np.cos(coord_table[origin,i,j,1]) * (b[0]*dSH0_theta + b[1]*dSH1_theta + b[2]*dSH2_theta)
                    x1 += -np.sin(coord_table[origin,i,j,1]) / np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_phi + b[2]*dSH2_phi)
                    
                    y1 += np.sin(coord_table[origin,i,j,0]) * np.sin(coord_table[origin,i,j,1]) * (b[0]*dSH0_r + b[1]*dSH1_r + b[2]*dSH2_r)
                    y1 += np.cos(coord_table[origin,i,j,0]) * np.sin(coord_table[origin,i,j,1]) * (b[0]*dSH0_theta + b[1]*dSH1_theta + b[2]*dSH2_theta)
                    y1 += np.cos(coord_table[origin,i,j,1]) / np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_phi + b[2]*dSH2_phi)
                    
                    z1 += np.cos(coord_table[origin,i,j,0]) * (b[0]*dSH0_r + b[1]*dSH1_r + b[2]*dSH2_r)
                    z1 += -np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_theta + b[1]*dSH1_theta + b[2]*dSH2_theta)
                    
                    for k in range(N*M):
                        j1, k1 = k//M, k%M
                        l = get_l(k1)
                        m = get_m(k1)
                        ind = int(l*(l+1)/2+np.abs(m))
                        dSH_theta, dISH_theta = 0, 0
                        dSH_phi, dISH_phi = 0, 0
                        dSH_r, dISH_r = 0, 0
                        if m >= 0:
                            dSH_theta += m*spherical_harmonics[j1,i,j,ind] / np.tan(coord_table[j1,i,j,0])
                            dISH_theta += m*image_spherical_harmonics[j1,i,j,ind] / np.tan(image_coord_table[j1,i,j,0])
                            dSH_phi += 1j*m*spherical_harmonics[j1,i,j,ind]
                            dISH_phi += 1j*m*image_spherical_harmonics[j1,i,j,ind]
                            dSH_r += spherical_harmonics[j1,i,j,ind]
                            dISH_r += image_spherical_harmonics[j1,i,j,ind]
                            if m != l:
                                dSH_theta += np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[j1,i,j,1]) * spherical_harmonics[j1,i,j,ind+1]
                                dISH_theta += np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*image_coord_table[j1,i,j,1]) * image_spherical_harmonics[j1,i,j,ind+1]
                        else:
                            dSH_theta += (-1)**m*m * np.conj(spherical_harmonics[j1,i,j,ind]) / np.tan(coord_table[j1,i,j,0]) + (-1)**m*np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[j1,i,j,1]) * np.conj(spherical_harmonics[j1,i,j,ind-1])
                            dISH_theta += (-1)**m*m * np.conj(image_spherical_harmonics[j1,i,j,ind]) / np.tan(image_coord_table[j1,i,j,0]) + (-1)**m*np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*image_coord_table[j1,i,j,1]) * np.conj(image_spherical_harmonics[j1,i,j,ind-1])
                            dSH_phi += (-1)**m*1j*m * np.conj(spherical_harmonics[j1,i,j,ind])
                            dISH_phi += (-1)**m*1j*m * np.conj(image_spherical_harmonics[j1,i,j,ind])
                            dSH_r += (-1)**m * np.conj(spherical_harmonics[j1,i,j,ind])
                            dISH_r += (-1)**m * np.conj(image_spherical_harmonics[j1,i,j,ind])
                            
                        x1 += (l+1)*A[k]*np.sin(coord_table[j1,i,j,0]) * np.cos(coord_table[j1,i,j,1]) * coord_table[j1,i,j,3]**(-l-2)*dSH_r
                        x1 += (l+1)*A[k]*np.sin(image_coord_table[j1,i,j,0]) * np.cos(image_coord_table[j1,i,j,1]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_r
                        
                        x1 += -A[k]*np.cos(coord_table[j1,i,j,0]) * np.cos(coord_table[j1,i,j,1]) * coord_table[j1,i,j,3]**(-l-2)*dSH_theta
                        x1 += -A[k]*np.cos(image_coord_table[j1,i,j,0]) * np.cos(image_coord_table[j1,i,j,1]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_theta
                        
                        
                        x1 += A[k]*np.sin(coord_table[j1,i,j,1]) / np.sin(coord_table[j1,i,j,0]) * coord_table[j1,i,j,3]**(-l-2)*dSH_phi
                        x1 += A[k]*np.sin(image_coord_table[j1,i,j,1]) / np.sin(image_coord_table[j1,i,j,0]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_phi
                        
                        
                        y1 += (l+1)*A[k]*np.sin(coord_table[j1,i,j,0]) * np.sin(coord_table[j1,i,j,1]) * coord_table[j1,i,j,3]**(-l-2)*dSH_r
                        y1 += (l+1)*A[k]*np.sin(image_coord_table[j1,i,j,0]) * np.sin(image_coord_table[j1,i,j,1]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_r
                        
                        y1 += -A[k]*np.cos(coord_table[j1,i,j,0]) * np.sin(coord_table[j1,i,j,1]) * coord_table[j1,i,j,3]**(-l-2)*dSH_theta
                        y1 += -A[k]*np.cos(image_coord_table[j1,i,j,0]) * np.sin(image_coord_table[j1,i,j,1]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_theta
                        
                        y1 += -A[k]*np.cos(coord_table[j1,i,j,1]) / np.sin(coord_table[j1,i,j,0]) * coord_table[j1,i,j,3]**(-l-2)*dSH_phi
                        y1 += -A[k]*np.cos(image_coord_table[j1,i,j,1]) / np.sin(image_coord_table[j1,i,j,0]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_phi
                    
                        
                        z1 += (l+1)*A[k]*np.cos(coord_table[j1,i,j,0]) * coord_table[j1,i,j,3]**(-l-2)*dSH_r
                        z1 += (l+1)*A[k]*np.cos(image_coord_table[j1,i,j,0]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_r
                        
                        z1 += A[k]*np.sin(coord_table[j1,i,j,0]) * coord_table[j1,i,j,3]**(-l-2)*dSH_theta
                        z1 += A[k]*np.sin(image_coord_table[j1,i,j,0]) * (-1)**(l+m)*beta*image_coord_table[j1,i,j,3]**(-l-2) * dISH_theta
                        
                        
                else:
                    x1 += np.sin(coord_table[origin,i,j,0]) * np.cos(coord_table[origin,i,j,1]) * (b[0]*dSH0_r + b[1]*dSH1_r*epsilon_plus/epsilon_minus + b[2]*dSH2_r)
                    x1 += np.cos(coord_table[origin,i,j,0]) * np.cos(coord_table[origin,i,j,1]) * (b[0]*dSH0_theta + b[1]*dSH1_theta*epsilon_plus/epsilon_minus + b[2]*dSH2_theta)
                    x1 += -1*np.sin(coord_table[origin,i,j,1]) / np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_phi + b[2]*dSH2_phi)
                    
                    y1 += np.sin(coord_table[origin,i,j,0]) * np.sin(coord_table[origin,i,j,1]) * (b[0]*dSH0_r + b[1]*dSH1_r*epsilon_plus/epsilon_minus + b[2]*dSH2_r)
                    y1 += np.cos(coord_table[origin,i,j,0]) * np.sin(coord_table[origin,i,j,1]) * (b[0]*dSH0_theta + b[1]*dSH1_theta*epsilon_plus/epsilon_minus + b[2]*dSH2_theta)
                    y1 += np.cos(coord_table[origin,i,j,1]) / np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_phi + b[2]*dSH2_phi)
                    
                    z1 += np.cos(coord_table[origin,i,j,0]) * (b[0]*dSH0_r + b[1]*dSH1_r*epsilon_plus/epsilon_minus + b[2]*dSH2_r)
                    z1 += -1*np.sin(coord_table[origin,i,j,0]) * (b[0]*dSH0_theta + b[1]*dSH1_theta*epsilon_plus/epsilon_minus + b[2]*dSH2_theta)
                    for k in range(N*M):
                        j1, k1 = k//M, k%M
                        l = get_l(k1)
                        m = get_m(k1)
                        ind = int(l*(l+1)/2+np.abs(m))
                        dSH_theta = 0
                        dSH_phi = 0
                        dSH_r = 0
                        if m >= 0:
                            dSH_theta += m*spherical_harmonics[j1,i,j,ind] / np.tan(coord_table[j1,i,j,0])
                            dSH_phi += 1j*m*spherical_harmonics[j1,i,j,ind]
                            dSH_r += spherical_harmonics[j1,i,j,ind]
                            if m != l:
                                dSH_theta += np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[j1,i,j,1]) * spherical_harmonics[j1,i,j,ind+1]
                        else:
                            dSH_theta += (-1)**m*m * np.conj(spherical_harmonics[j1,i,j,ind]) / np.tan(coord_table[j1,i,j,0]) + (-1)**m*np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[j1,i,j,1]) * np.conj(spherical_harmonics[j1,i,j,ind-1])
                            dSH_phi += (-1)**m*1j*m * np.conj(spherical_harmonics[j1,i,j,ind])
                            dSH_r += (-1)**m * np.conj(spherical_harmonics[j1,i,j,ind])
                            
                        x1 += (l+1)*beta2*A[k] * np.sin(coord_table[j1,i,j,0]) * np.cos(coord_table[j1,i,j,1]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_r)
                        x1 += -beta2*A[k] * np.cos(coord_table[j1,i,j,0]) * np.cos(coord_table[j1,i,j,1]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_theta)
                        x1 += beta2*A[k] * np.sin(coord_table[j1,i,j,1]) / np.sin(coord_table[j1,i,j,0]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_phi)

                        y1 += (l+1)*beta2*A[k] * np.sin(coord_table[j1,i,j,0]) * np.sin(coord_table[j1,i,j,1]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_r)
                        y1 += -beta2*A[k] * np.cos(coord_table[j1,i,j,0]) * np.sin(coord_table[j1,i,j,1]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_theta)
                        y1 += -beta2*A[k] * np.cos(coord_table[j1,i,j,1]) / np.sin(coord_table[j1,i,j,0]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_phi)

                        z1 += (l+1)*beta2*A[k] * np.cos(coord_table[j1,i,j,0]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_r)
                        z1 += beta2*A[k] * np.sin(coord_table[j1,i,j,0]) * (coord_table[j1,i,j,3]**(-l-2)*dSH_theta)
                        
            else:
                for k in range(M):
                    l = get_l(k)
                    m = get_m(k)
                    ind = int(l*(l+1)/2+np.abs(m))
                    dSH_theta = 0
                    dSH_phi = 0
                    dSH_r = 0
                    if m >= 0:
                        dSH_theta += m*spherical_harmonics[sphere,i,j,ind] / np.tan(coord_table[sphere,i,j,0])
                        dSH_phi += 1j*m*spherical_harmonics[sphere,i,j,ind]
                        dSH_r += spherical_harmonics[sphere,i,j,ind]
                        if m != l:
                            dSH_theta += np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[sphere,i,j,1]) * spherical_harmonics[sphere,i,j,ind+1]
                    else:
                        dSH_theta += (-1)**m*m * np.conj(spherical_harmonics[sphere,i,j,ind]) / np.tan(coord_table[sphere,i,j,0]) + (-1)**m*np.sqrt((l-m)*(l+m+1)) * np.exp(-1j*coord_table[sphere,i,j,1]) * np.conj(spherical_harmonics[sphere,i,j,ind-1])
                        dSH_phi += (-1)**m*1j*m * np.conj(spherical_harmonics[sphere,i,j,ind])
                        dSH_r += (-1)**m * np.conj(spherical_harmonics[sphere,i,j,ind])
                        
                    x1 += -l*B[sphere*M+k] * coord_table[sphere,i,j,3]**(l-1)*dSH_r
                    y1 += -B[sphere*M+k] * coord_table[sphere,i,j,3]**(l-1)*dSH_theta
                    z1 += -B[sphere*M+k] * coord_table[sphere,i,j,3]**(l-1)*dSH_phi / np.sin(coord_table[sphere,i,j,0])
                    
            E[i, j] = np.sqrt(x1*np.conj(x1) + y1*np.conj(y1) + z1*np.conj(z1))
    return E

# --------------------------------------------------------------------------
# --- end implementing functions
# --------------------------------------------------------------------------

# =========================================================================
#  Main
# =========================================================================

#Returns a dictionary with the elements specified by the argument output.
#Parameters:
#
#output - array or list of strings to be used as dictionary keys
#for the output dictionary. Possible options are:
#"p_x" is the dimensinoless dipole moment with incident electric field 
#parallel to the x-axis. Similarly for "p_y" and "p_z". "p_c" has an 
#incident field spesified by the arguments theta_0 and phi_0. The 
#dipole moments have dimensions [N_cell, N_ohm].
#Likewise for the potentials "pot_x, "pot_y", "pot_z" and "pot_c",
#and the electric fields "E_x", "E_y", "E_z" and "E_c". The 
#potentials and fields have dimensions [N_ohm, N_x, N_z].
#Lastly, the reflectance "R_s" for s-polarized light and "R_p" for 
#p_polarised light can be computed.
#
#omega - array of energies in eV for the incident wave
#N_cell - number of spheres in the unit cell
#p - list of positions of sphere centres (x,y,z)
#r - corresponding relative radiuses for spheres in p
#L_parallel - multipole order of sphere interaction
#L_orthogonal - multipole order of substrate interaction
#epsilon_minus - dielectric function of the substrate.
#Can be a string with a material in the SOPRA database,
#a single value or an array of values corresponding to the
#incident wave energies in omega
#
#epsilon_plus - dielectric function of the ambient medium.
#Must be a single value
#
#epsilons - dielectric function of the spheres given as
#an array of dielectric values corresponding to the
#incident wave energies in omega. The dielectric values
#can be N_cell long arrays corresponding to the spheres
#in p, or a single value which will be applied to
#all the spheres.
#
#material - dielectric function of the spheres given as
#a string of the material from the SOPRA database. If
#only a single string is present it will be applied for
#all the spheres, otherwise an array of N_cell strings
#must be used corresponding to each sphere in p
#
#omega_p and gamma - the two parameters for the Drude
#model. If single values, they're applied for all spheres.
#Otherwise, an array of N_cell values must be submitted.
#
#grid_specs - an array of 7 parameters used for the
#grid where the potential and electric field is
#evaluated. First two are start and end x coordinates.
#Next two are start and end z coordinates. Next two
#are number of x and z points. Last is y coordinate.
#
#origin - which sphere to be used as an origin in
#the potential plots
#
#periodic - whether or not the unit cell shall be
#repeated periodically. If True, nearfield calculations
#are not calculated. If False, reflectances are not
#calculated.
#
#theta and phi - polar and azimuthal angles of
#the incident plane wave.
#
#theta_0 and phi_0 - polar and azimuthal angles of
#the electric field for custom calculations of
#dipole moment, potential and electric field 
#(p_c, pot_c and E_c).
#
#R_interaction - interaction distance. Only unit cells
#within this distance will be included in calculations
#
#length_scale - conversion for the sphere radiuses in r
#to metres
#b1_x and b1_y - the two lattice spacings between the
#unit cells
#
#alpha - the angle between b1_x and b1_y
#verbose - If computation times shall be printed
#freq - Number of energies between each printing
#tolerance - tolaerance for the iterative solver

def main(output, omega, N_cell, p, r, L_parallel, L_orthogonal, epsilon_minus, epsilon_plus = 1, epsilons = None, material = None, omega_p = None, gamma = None, grid_specs = None, origin = 0, periodic = False, theta = None, phi = None, theta_0 = None, phi_0 = None, R_interaction = None, length_scale = None, b1_x = None, b1_y = None, alpha = np.pi/2, verbose = True, freq = 10, tolerance = 1e-6):
    start1 = time()
    start0 = time()
    L = max(L_parallel, L_orthogonal) 
    M = (L+1)**2 - 1
    N_ohm = len(omega)
    
    #Control input parameters related to periodic boundary conditions
    if periodic:
        if np.any([theta == None, phi == None, R_interaction == None, length_scale == None, b1_x == None, b1_y == None, alpha == None]):
            print("Error: Some of the parameters theta, phi, R_interaction, length_scale, b1_x, b1_y and alpha are unspecified")
            return {}
        else:
            rho = N_cell/(b1_x*b1_y*np.sin(alpha))
            theta_p, phi_p, theta_s, phi_s = E_angles(theta, phi)
    else:
        b1_x = b1_y = 1000
        alpha = np.pi/2
        R_interaction = 1
        length_scale = 1
        theta = phi = 0
    
    
    #Prepare the dielectric functions
    if epsilons != None:
        if np.shape(epsilons) != (N_ohm, N_cell) and np.shape(epsilons) != (N_ohm,):
            print("Error: epsilons wrongly defined. Must either have dimensions (N_ohm, N_cell) or (N_ohm,).")
            return {}
        elif np.shape(epsilons) != (N_ohm,):
            epsilons = np.reshape(np.tile(epsilons, N_cell), [N_cell, N_ohm]).T
    elif material != None:
        if np.shape(material) == ():
            SOPRA_obj = EpsilonSOPRA(material)
            epsilons = SOPRA_obj.getepsilon(omega)
            epsilons = np.reshape(np.tile(epsilons, N_cell), [N_cell, N_ohm]).T
        elif np.shape(material) == (N_cell,):
            epsilons = np.zeros([N_cell, N_ohm], dtype = np.complex128)
            for i in range(N_cell):
                SOPRA_obj = EpsilonSOPRA(material[i])
                epsilons[i] = SOPRA_obj.getepsilon(omega)
            epsilons = epsilons.T
        else:
            print("Error: material wrongly defined. Must either be string or list/array of strings.")
            return {}
    elif omega_p != None and gamma != None:
        omega_p = np.array([omega_p]).flatten()
        gamma = np.array([gamma]).flatten()
        if len(omega_p) == 1 and len(gamma) == 1:
            epsilons = 1 - omega_p**2/(omega*(omega + 1j*gamma))
            epsilons = np.reshape(np.tile(epsilons, N_cell), [N_cell, N_ohm]).T
        elif len(omega_p) == N_cell and len(gamma) == N_cell:
            epsilons = np.zeros([N_cell, N_ohm])
            for i in range(N_cell):
                epsilons[i] = 1 - omega_p[i]**2/(omega*(omega + 1j*gamma[i]))
            epsilons = epsilons.T
        else:
            print("Error: Mismatch in length of omega_p and gamma")
            return {}
    else:
        print("Error: Dielectric function must be defined either by epsilons, material or omega_p and gamma.")
        return {}
    
    if type(epsilon_minus) == str:
        SOPRA_obj = EpsilonSOPRA(epsilon_minus)
        epsilon_minus = SOPRA_obj.getepsilon(omega)
    else:
        if type(epsilon_minus) != np.ndarray:
            epsilon_minus = np.array([epsilon_minus]).flatten()
        if len(epsilon_minus) != N_ohm and len(epsilon_minus) != 1:
            print("Error: epsilon_minus must either be  a string of a material, a number or a list/array of numbers with length N_ohm.")
            return {}
        elif len(epsilon_minus) == 1:
            epsilon_minus = epsilon_minus*np.ones(N_ohm, dtype = np.complex128)  
    
    beta = (epsilon_plus - epsilon_minus)/(epsilon_plus + epsilon_minus)
    beta2 = 2*epsilon_plus/(epsilon_plus + epsilon_minus)
    
    
    #Allocate memory
    contours = np.array(["pot_x", "pot_y", "pot_z", "pot_c", "E_x", "E_y", "E_z", "E_c"])
    if np.any(np.isin(output, contours)):
        if not periodic:
            if grid_specs != None:
                if grid_specs[5]*grid_specs[6] > 1e4:
                    print("Runtime warning: Nearfield calculations with high resolution has long initialisation time for the spherical harmonics.")

                table = np.zeros([N_cell, grid_specs[5], grid_specs[4], 5], dtype = np.complex128)
                x, z, grid_coords, grid_image_coords = coord_table(table, p, N_cell, grid_specs[0], grid_specs[1], grid_specs[2], grid_specs[3], grid_specs[4], grid_specs[5], grid_specs[6])
                grid_spherical_harmonics, grid_image_spherical_harmonics = spherical_harm(grid_coords, grid_image_coords, L, N_cell, grid_specs[4], grid_specs[5])

                if np.isin("pot_x", output):
                    pot_x = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("pot_y", output):
                    pot_y = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("pot_z", output):
                    pot_z = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("pot_c", output):
                    pot_c = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)

                if np.isin("E_x", output):
                    E_x = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("E_y", output):
                    E_y = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("E_z", output):
                    E_z = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
                if np.isin("E_c", output):
                    E_c = np.zeros([N_ohm, grid_specs[5], grid_specs[4]], dtype = np.complex128)
            else:
                print("Error: grid_specs undefined")
                return {}
        else:
            print("Error: Nearfield calculations only works for finite systems in this code. Set finite to True.")
            output = np.setdiff1d(output, contours)
            
    dipole_moments = np.array(["p_x", "p_y", "p_z", "p_c"])
    if np.any(np.isin(output, dipole_moments)):
        if np.isin("p_x", output):
            p_x = np.zeros([N_cell, N_ohm])
        if np.isin("p_y", output):
            p_y = np.zeros([N_cell, N_ohm])
        if np.isin("p_z", output):
            p_z = np.zeros([N_cell, N_ohm])
        if np.isin("p_c", output):
            p_c = np.zeros([N_cell, N_ohm])

    reflectances = np.array(["R_s", "R_p"])
    if np.any(np.isin(output, reflectances)):
        if periodic:
            if np.isin("R_p", output):
                R_p = np.zeros(N_ohm, dtype = np.complex128)
            if np.isin("R_s", output):
                R_s = np.zeros(N_ohm, dtype = np.complex128)
        else:
            print("Error: The calculations must be done for a periodic infinite system in order to determine reflectances. Set 'finite' to False.")
            output = np.setdiff1d(output, reflectances)
            
    computations = np.array(["p_x", "p_y", "p_z", "p_c", "R_s", "R_p", "pot_x", "pot_y", "pot_z", "pot_c", "E_x", "E_y", "E_z", "E_c"])      
    if not np.any(np.isin(output, computations)):
        print("No optional computations specified")
        return {}
    
    
    #Compute values independent of the dielectric functions of the spheres:
    #These are faster to just look up than to compute each time, and do 
    #only have to be computed once. Numba is also not as cooperative with
    #pyshtools and binomial coefficients.
    
    #coords[i, j, p, k] is the kth coordinate from get_polar_coords
    #of sphere j  at lattice point p relative to sphere i in centre 
    #lattice point. The lattice point's corrdinates are retrieved by 
    #lattice_vectors[p].
    
    #SHTOOLS:
    #spherical_harmonics[i, j, p] is an array of all the spherical harmonics
    #for sphere j at lattice point p relative to sphere i in the centre 
    #lattice point. This is on compact form so the
    #elements are indexed by ind = int(l*(l+1)/2+np.abs(m)),
    #such that only for m>=0 are contained.
    
    lattice_points = get_length(R_interaction, b1_x, b1_y, alpha)
    coords = np.zeros([N_cell, N_cell, lattice_points, 5])
    spherical_harmonics = np.zeros([N_cell, N_cell, lattice_points, (2*L+2)*(2*L+1)//2], dtype = np.complex128)
    image_spherical_harmonics = np.zeros([N_cell, N_cell, lattice_points, (2*L+2)*(2*L+1)//2], dtype = np.complex128)
    lattice_vectors = np.zeros([lattice_points, 2])
    
    lattice_point = 0
    centre_point = 0
    j1s = get_j1(R_interaction, b1_y, alpha)
    for j1 in j1s:
        i1s = get_i1(R_interaction, b1_x, b1_y, alpha, j1)
        for i1 in i1s:
            lattice_vectors[lattice_point] = np.array([i1*b1_x + j1*b1_y*np.cos(alpha), j1*b1_y*np.sin(alpha)])*r[0]
            if j1 == 0 and i1 == 0:
                centre_point = lattice_point
            for j in range(N_cell):
                for i in range(N_cell):
                    coords[j,i,lattice_point] = get_spherical_coords(p[j], p[i]+np.concatenate((lattice_vectors[lattice_point], np.array([0]))))
                    spherical_harmonics[j,i,lattice_point] = expand.spharm(2*L, coords[j,i,lattice_point,0], coords[j,i,lattice_point,1], normalization = 'ortho', kind = 'complex', csphase = -1, packed = True, degrees = False)[0]
                    image_spherical_harmonics[j,i,lattice_point] = expand.spharm(2*L, coords[j,i,lattice_point,2], coords[j,i,lattice_point,1], normalization = 'ortho', kind = 'complex', csphase = -1, packed = True, degrees = False)[0]
            lattice_point += 1
    
    #Compute H(l_j, m_j | l_i, m_i) for all the M*M outcomes 
    #and store them in a table indexed by k and l
    Hs = np.reshape(H(np.arange(M**2), M), [M, M])

    
    end0 = time() 
    if verbose:
        print("Initialization complete. Excecution time taken: ", end0-start0)
        print("Energy number:  Excecution time [s]:  Total expected excecution time [hrs]: ")
        
    #First energy, LU-factorisation is used to solve for A
    k = get_k(omega[0], theta, phi, length_scale)
    C = np.zeros([M*N_cell, M*N_cell], dtype = np.complex128)
    C = get_C(C, L_parallel, L_orthogonal, M, N_cell, lattice_points, centre_point, lattice_vectors, epsilons[0], r, beta[0], epsilon_plus, coords, spherical_harmonics, image_spherical_harmonics, Hs, k, b1_x, b1_y, alpha)

    
    x_computations = np.array(["p_x", "pot_x", "E_x"])
    x_nearfields = np.array(["pot_x", "E_x"])
    if np.any(np.isin(output, x_computations)):
        b_x = get_b(np.pi/2, 0, M, N_cell) 
        A_x = np.linalg.solve(C, b_x) 
        A_x_prev = A_x.copy() 
            
        if np.isin("p_x", output):
            for n in range(N_cell): 
                p_x[n,0] = get_p(A_x, n, M, r)
                
        if np.any(np.isin(x_nearfields, output)):
            B_x = get_B(M, N_cell, A_x, b_x, beta[0], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
            if np.isin("pot_x", output):
                B0_x = get_B_0(M, N_cell, A_x, beta[0], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_x)
                pot_x[0] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_x, r, N_cell, M, beta[0], b_x, p, B_x, beta2[0], epsilon_plus, epsilon_minus[0], B0_x, np.pi/2)
            if np.isin("E_x", output):
                E_x[0] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_x, r, N_cell, M, beta[0], b_x, p, B_x, beta2[0], epsilon_plus, epsilon_minus[0])
       
    
    y_computations = np.array(["p_y", "pot_y", "E_y"])
    y_nearfields = np.array(["pot_y", "E_y"])
    if np.any(np.isin(output, y_computations)):
        b_y = get_b(np.pi/2, np.pi/2, M, N_cell)       
        A_y = np.linalg.solve(C, b_y) 
        A_y_prev = A_y.copy() 
        
        if np.isin("p_y", output):
            for n in range(N_cell): 
                p_y[n,0] = get_p(A_y, n, M, r)

        if np.any(np.isin(y_nearfields, output)):
            B_y = get_B(M, N_cell, A_y, b_y, beta[0], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
            if np.isin("pot_y", output):
                B0_y = get_B_0(M, N_cell, A_y, beta[0], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_y)
                pot_y[0] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_y, r, N_cell, M, beta[0], b_y, p, B_y, beta2[0], epsilon_plus, epsilon_minus[0], B0_y, np.pi/2)
            if np.isin("E_x", output):
                E_y[0] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_y, r, N_cell, M, beta[0], b_y, p, B_y, beta2[0], epsilon_plus, epsilon_minus[0])
  

    z_computations = np.array(["p_z", "pot_z", "E_z"])
    z_nearfields = np.array(["pot_z", "E_z"])         
    if np.any(np.isin(output, z_computations)):
        b_z = get_b(np.pi, 0, M, N_cell)     
        A_z = np.linalg.solve(C, b_z) 
        A_z_prev = A_z.copy() 
        
        if np.isin("p_z", output):
            for n in range(N_cell): 
                p_z[n,0] = get_p(A_z, n, M, r)
         
        if np.any(np.isin(z_nearfields, output)):
            B_z = get_B(M, N_cell, A_z, b_z, beta[0], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
            if np.isin("pot_z", output):
                B0_z = get_B_0(M, N_cell, A_z, beta[0], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_z)
                pot_z[0] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_z, r, N_cell, M, beta[0], b_z, p, B_z, beta2[0], epsilon_plus, epsilon_minus[0], B0_z, np.pi)
            if np.isin("E_z", output):
                E_z[0] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_z, r, N_cell, M, beta[0], b_z, p, B_z, beta2[0], epsilon_plus, epsilon_minus[0])
                
                
    c_computations = np.array(["p_c", "pot_c", "E_c"])
    c_nearfields = np.array(["pot_c", "E_c"])
    if np.any(np.isin(output, c_computations)):                
        if theta_0 != None and phi_0 != None:
            b_c = get_b(theta_0, phi_0, M, N_cell)       
            A_c = np.linalg.solve(C, b_c) 
            A_c_prev = A_c.copy() 
        
            if np.isin("p_c", output):
                for n in range(N_cell): 
                    p_c[n,0] = get_p(A_c, n, M, r)
        
            if np.any(np.isin(c_nearfields, output)):
                B_c = get_B(M, N_cell, A_c, b_c, beta[0], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
                if np.isin("pot_c", output):
                    B0_c = get_B_0(M, N_cell, A_c, beta[0], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_y)
                    pot_c[0] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_c, r, N_cell, M, beta[0], b_c, p, B_c, beta2[0], epsilon_plus, epsilon_minus[0], B0_c, theta_0)
                if np.isin("E_c", output):
                    E_c[0] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_c, r, N_cell, M, beta[0], b_c, p, B_c, beta2[0], epsilon_plus, epsilon_minus[0])
                    
        else:
            print("Error: theta_0 and phi_0 must be specified for custom angle input computations.")
            output = np.setdiff1d(output, c_computations)
    
    
    if np.any(np.isin(output, reflectances)):
        if np.isin("R_s", output):
            b_s = get_b(theta_s, phi_s, M, N_cell)      
            A_s = np.linalg.solve(C, b_s)
            A_s_prev = A_s.copy()
            gamma_s, beta_s = susceptibilities(A_s, M, theta_s, phi_s, epsilon_plus, p[0,2], rho)
            R_s[0] = s_polarization(theta, omega[0], gamma_s, epsilon_plus, epsilon_minus[0], length_scale)[0]

        if np.isin("R_p", output):
            b_p = get_b(theta_p, phi_p, M, N_cell) #Build B-vector      
            A_p = np.linalg.solve(C, b_p) #Solve for A
            A_p_prev = A_p.copy() #Copy solution to start iteration
            gamma_p, beta_p = susceptibilities(A_p, M, theta_p, phi_p, epsilon_plus, p[0,2], rho)
            R_p[0] = p_polarization(theta, omega[0], gamma_p, epsilon_plus, epsilon_minus[0], beta_p, length_scale)[0]
        

    #For the rest of the energies an iterative solver is used
    for o in range(1, N_ohm):
        start2 = time()
        k = get_k(omega[o], theta, phi, length_scale)
        C = np.zeros([M*N_cell, M*N_cell], dtype = np.complex128)
        C = get_C(C, L_parallel, L_orthogonal, M, N_cell, lattice_points, centre_point, lattice_vectors, epsilons[o], r, beta[o], epsilon_plus, coords, spherical_harmonics, image_spherical_harmonics, Hs, k, b1_x, b1_y, alpha)


        if np.any(np.isin(output, x_computations)):
            A_x, info = gmres(C, b_x, A_x_prev, tol = tolerance) 
            A_x_prev = A_x.copy() 

            if np.isin("p_x", output):
                for n in range(N_cell): 
                    p_x[n,o] = get_p(A_x, n, M, r)

            if np.any(np.isin(x_nearfields, output)):
                B_x = get_B(M, N_cell, A_x, b_x, beta[o], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
                if np.isin("pot_x", output):
                    B0_x = get_B_0(M, N_cell, A_x, beta[o], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_x)
                    pot_x[o] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_x, r, N_cell, M, beta[o], b_x, p, B_x, beta2[o], epsilon_plus, epsilon_minus[o], B0_x, np.pi/2)
                if np.isin("E_x", output):
                    E_x[o] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_x, r, N_cell, M, beta[o], b_x, p, B_x, beta2[o], epsilon_plus, epsilon_minus[o])


        if np.any(np.isin(output, y_computations)):
            A_y, info = gmres(C, b_y, A_y_prev, tol = tolerance) 
            A_y_prev = A_y.copy() 

            if np.isin("p_y", output):
                for n in range(N_cell): 
                    p_y[n,o] = get_p(A_y, n, M, r)

            if np.any(np.isin(y_nearfields, output)):
                B_y = get_B(M, N_cell, A_y, b_y, beta[o], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
                if np.isin("pot_y", output):
                    B0_y = get_B_0(M, N_cell, A_y, beta[o], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_y)
                    pot_y[o] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_y, r, N_cell, M, beta[o], b_y, p, B_y, beta2[o], epsilon_plus, epsilon_minus[o], B0_y, np.pi/2)
                if np.isin("E_x", output):
                    E_y[o] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_y, r, N_cell, M, beta[o], b_y, p, B_y, beta2[o], epsilon_plus, epsilon_minus[o])


        if np.any(np.isin(output, z_computations)):
            A_z, info = gmres(C, b_z, A_z_prev, tol = tolerance) 
            A_z_prev = A_z.copy() 

            if np.isin("p_z", output):
                for n in range(N_cell): 
                    p_z[n,o] = get_p(A_z, n, M, r)

            if np.any(np.isin(z_nearfields, output)):
                B_z = get_B(M, N_cell, A_z, b_z, beta[o], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
                if np.isin("pot_z", output):
                    B0_z = get_B_0(M, N_cell, A_z, beta[o], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_z)
                    pot_z[o] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_z, r, N_cell, M, beta[o], b_z, p, B_z, beta2[o], epsilon_plus, epsilon_minus[o], B0_z, np.pi)
                if np.isin("E_z", output):
                    E_z[o] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_z, r, N_cell, M, beta[o], b_z, p, B_z, beta2[o], epsilon_plus, epsilon_minus[o])


        if np.any(np.isin(output, c_computations)):                
            A_c, info = gmres(C, b_c, A_c_prev, tol = tolerance) 
            A_c_prev = A_c.copy() 

            if np.isin("p_c", output):
                for n in range(N_cell): 
                    p_c[n,o] = get_p(A_c, n, M, r)

            if np.any(np.isin(c_nearfields, output)):
                B_c = get_B(M, N_cell, A_c, b_c, beta[o], r, Hs, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal)
                if np.isin("pot_c", output):
                    B0_c = get_B_0(M, N_cell, A_c, beta[o], r, coords[:,:,centre_point,:], spherical_harmonics[:,:,centre_point], image_spherical_harmonics[:,:,centre_point], L_parallel, L_orthogonal, origin, B_y)
                    pot_c[o] = potential_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_c, r, N_cell, M, beta[o], b_c, p, B_c, beta2[o], epsilon_plus, epsilon_minus[o], B0_c, theta_0)
                if np.isin("E_c", output):
                    E_c[o] = field_grid(np.zeros([grid_specs[5], grid_specs[4]], dtype = np.complex128), origin, x, grid_specs[6], z, grid_specs[5], grid_specs[4], grid_coords, grid_image_coords, grid_spherical_harmonics, grid_image_spherical_harmonics, A_c, r, N_cell, M, beta[o], b_c, p, B_c, beta2[o], epsilon_plus, epsilon_minus[o])


        if np.any(np.isin(output, reflectances)):
            if np.isin("R_s", output):
                A_s, info = gmres(C, b_s, A_s_prev, tol = tolerance)
                A_s_prev = A_s.copy()
                gamma_s, beta_s = susceptibilities(A_s, M, theta_s, phi_s, epsilon_plus, p[0,2], rho)
                R_s[o] = s_polarization(theta, omega[o], gamma_s, epsilon_plus, epsilon_minus[o], length_scale)[0]

            if np.isin("R_p", output):
                A_p, info = gmres(C, b_p, A_p_prev, tol = tolerance)
                A_p_prev = A_p.copy()
                gamma_p, beta_p = susceptibilities(A_p, M, theta_p, phi_p, epsilon_plus, p[0,2], rho)
                R_p[o] = p_polarization(theta, omega[o], gamma_p, epsilon_plus, epsilon_minus[o], beta_p, length_scale)[0]


        end2 = time()
        if verbose and o % freq == 0:
            print(o, end2-start2, (end2-start2)*N_ohm/3600)
            
            
    #Creating the output dictionary
    dict_out = {}
    
    if np.any(np.isin(output, contours)):
        if np.isin("pot_x", output):
            dict_out["pot_x"] = pot_x
        if np.isin("pot_y", output):
            dict_out["pot_y"] = pot_y
        if np.isin("pot_z", output):
            dict_out["pot_z"] = pot_z
        if np.isin("pot_c", output):
            dict_out["pot_c"] = pot_c

        if np.isin("E_x", output):
            dict_out["E_x"] = E_x
        if np.isin("E_y", output):
            dict_out["E_y"] = E_y
        if np.isin("E_z", output):
            dict_out["E_z"] = E_z
        if np.isin("E_c", output):
            dict_out["E_c"] = E_c
                              
        if np.isin("x", output):
            dict_out["x"] = x
        if np.isin("z", output):
            dict_out["z"] = z
    
    if np.any(np.isin(output, dipole_moments)):
        if np.isin("p_x", output):
            dict_out["p_x"] = p_x
        if np.isin("p_y", output):
            dict_out["p_y"] = p_y
        if np.isin("p_z", output):
            dict_out["p_z"] = p_z
        if np.isin("p_c", output):
            dict_out["p_c"] = p_c
            
    if np.any(np.isin(output, reflectances)):
        if np.isin("R_p", output):
            dict_out["R_p"] = np.absolute(R_p)**2
        if np.isin("R_s", output):
            dict_out["R_s"] = np.absolute(R_s)**2
            
    if np.isin("epsilon",output):
        dict_out["epsilon"] = epsilons
    
    
    end1 = time()
    if verbose:
        print("Task complete: ", end1-start1)
    return dict_out

# --------------------------------------------------------------------------
# --- end main
# --------------------------------------------------------------------------



In [1]:
# =========================================================================
#  Examples
# =========================================================================

h = 0.01
d = 0.2
N_cell = 2
r = np.ones(N_cell)
p2 = np.array([[0,0,1+h], [2+d,0,1+h]], dtype = np.float64)
omega = np.linspace(2.6, 3.8, 121)
gridspecs = (-1.4499, 3.5501, -1.4499, 3.5501, 31, 31, 0)

output1 = np.array(["R_s", "R_p"])
output2 = np.array(["pot_x", "E_x"])

out1 = main(output1, omega, 1, p2, r, 10, 10, epsilon_minus = 2.76, material = "ag", periodic = True, theta = np.pi/4, phi = 0, R_interaction = 30, length_scale = 1e-8, b1_x = 2.2, b1_y = 2.2)
out2 = main(output2, omega, N_cell, p2, r, 10, 10, epsilon_minus = 2.76, material = "ag", periodic = False, grid_specs = gridspecs)

# --------------------------------------------------------------------------
# --- end examples
# --------------------------------------------------------------------------


NameError: name 'np' is not defined

In [4]:
import os
# os.chdir('C:\\Users\\Xx-Fr\\pysopra1')
from pysopra2 import EpsilonSOPRA, micron2eV, eV2micron

ModuleNotFoundError: No module named 'pysopra2'

In [2]:
# import os
# os.chdir('C:\\Users\\Xx-Fr\\pysopra-0.0.1\\pysopra')